In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
pd.options.display.float_format = '{:.5f}'.format
bus_bts = pd.read_csv("bus_bts.csv")
train = pd.read_csv("train.csv")
submission = pd.read_csv("submission_sample.csv")
rain = pd.read_csv("rain.csv",encoding='euc-kr')

In [2]:
rain.columns = ['지점', '지점명', '일시', '강수량', 'QC']

In [3]:
#오류인 관측치를 0으로 대체
rain2 = rain.fillna(0)

In [4]:
#일시를 기준으로 정렬
rain2 = rain2.sort_values(by='일시')
rain2 = rain2.reset_index()

In [5]:
#6시~12시 관측값만 남기기
rain2 = rain2[rain2['일시'].str.contains('07:|08:|09:|10:|11:|12:')]

In [6]:
#동시간 네지역 강수량을 합치기
for i in range (0,len(rain2)-1) :
    if (rain2.iloc[i,3] == rain2.iloc[i+1, 3]) :
        rain2.iloc[i+1,4] += rain2.iloc[i,4]
        
rain2 = rain2.drop_duplicates(['일시'])

In [7]:
#6시~12시 강수량을 일별로 모두 합치기
day_rain = rain2
for i in range (0,len(day_rain)-1) :
    if i%6==5 :
        continue
    else :
        day_rain.iloc[i+1,4] += day_rain.iloc[i,4]
        
day_rain = day_rain.reset_index()
day_rain = day_rain.drop(['index'],axis=1)

for i in range(0,len(day_rain)-1) :
    if i%6==5:
        continue
    else:
        day_rain = day_rain.drop(index=i,axis=0)

day_rain = day_rain.reset_index()

In [8]:
day_rain["일시"] = pd.to_datetime(day_rain.일시,format='%Y-%m-%d %H:%M')
day_rain["일시"]= day_rain["일시"].dt.date
day_rain = day_rain.drop(['index','level_0','지점명','지점','QC'], axis=1)

In [9]:
#비 안옴(0), 약한 비(1), 강한 비(2) 로 구분
rain_class = []

for i in range (len(day_rain)):
    if (day_rain.iloc[i,1] == 0):
        rain_class.append(0)
    elif (day_rain.iloc[i,1] <= 18) :
        rain_class.append(1)
    elif (day_rain.iloc[i,1] > 18):
        rain_class.append(2)

day_rain["강수 정도"] = rain_class

In [10]:
pd.get_dummies(day_rain, columns=['강수 정도'])

,일시,강수량,강수 정도_0,강수 정도_1,강수 정도_2
0,2019-09-01,0.00000,1,0,0
1,2019-09-02,15.30000,0,1,0
2,2019-09-03,6.40000,0,1,0
3,2019-09-04,80.60000,0,0,1
4,2019-09-05,1.70000,0,1,0
5,2019-09-06,2.10000,0,1,0
6,2019-09-07,0.00000,1,0,0
7,2019-09-08,0.30000,0,1,0
8,2019-09-09,10.20000,0,1,0
9,2019-09-11,0.00000,1,0,0


In [11]:
mount = []
for i in range(len(train.latitude)) :
    if ((126.35 < train.iloc[i,7] < 126.72) & (33.3 < train.iloc[i,6] < 33.44)) :
        mount.append(1)
    else :
        mount.append(0)
        
mount = pd.DataFrame(mount)
train = pd.concat([mount,train],axis=1)
train.rename(columns={0:'산지'}, inplace=True)

In [13]:
train

,산지,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.00000,...,5.00000,2.00000,6.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.00000,...,2.00000,5.00000,6.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.00000
2,0,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.00000,...,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000
3,0,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.00000,...,26.00000,14.00000,16.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,53.00000
4,0,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415418,0,415418,2019-09-30,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,4.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
415419,0,415419,2019-09-30,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,4.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
415420,0,415420,2019-09-30,32820000,시내,2322,해병부대,33.23100,126.26273,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000
415421,0,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,1.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [14]:
pd.get_dummies(train, columns=['산지'])

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,산지_0,산지_1
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.00000,1.00000,...,6.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.00000,4.00000,...,6.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.00000,1,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.00000,1.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000,1,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.00000,17.00000,...,16.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,53.00000,1,0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415418,415418,2019-09-30,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,4.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0
415419,415419,2019-09-30,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,4.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0
415420,415420,2019-09-30,32820000,시내,2322,해병부대,33.23100,126.26273,0.00000,0.00000,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0
415421,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,1.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0


In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train['6~7_ride'] = scaler.fit_transform(train[['6~7_ride']])
train['7~8_ride'] = scaler.fit_transform(train[['7~8_ride']])
train['8~9_ride'] = scaler.fit_transform(train[['8~9_ride']])
train['9~10_ride'] = scaler.fit_transform(train[['9~10_ride']])
train['10~11_ride'] = scaler.fit_transform(train[['10~11_ride']])
train['11~12_ride'] = scaler.fit_transform(train[['11~12_ride']])
train['6~7_takeoff'] = scaler.fit_transform(train[['6~7_takeoff']])
train['7~8_takeoff'] = scaler.fit_transform(train[['7~8_takeoff']])
train['8~9_takeoff'] = scaler.fit_transform(train[['8~9_takeoff']])
train['9~10_takeoff'] = scaler.fit_transform(train[['9~10_takeoff']])
train['10~11_takeoff'] = scaler.fit_transform(train[['10~11_takeoff']])
train['11~12_takeoff'] = scaler.fit_transform(train[['11~12_takeoff']])

In [25]:
train

,산지,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.00000,...,0.06410,0.01613,0.06061,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,0.01176,...,0.02564,0.04032,0.06061,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.00000
2,0,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,0.01176,...,0.02564,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000
3,0,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.00000,...,0.33333,0.11290,0.16162,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,53.00000
4,0,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01538,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415418,0,415418,2019-09-30,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,0.04706,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
415419,0,415419,2019-09-30,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,0.04706,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
415420,0,415420,2019-09-30,32820000,시내,2322,해병부대,33.23100,126.26273,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.01515,0.00000,0.00000,0.00000,0.00000,0.00000
415421,0,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,0.01176,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [30]:
test = pd.read_csv("test.csv")

In [ ]:
train = pd.get_dummies(train, columns=["in_out"], drop_first=True)

train = train.drop(['date', 'station_name'], axis=1)